In [12]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "resnet20_quant4bit"
model = resnet20_quant()
#model.conv1 = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.bn1 = bn1 = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.layer1[0].conv1 = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.layer1[0].conv2 = QuantConv2d(8, 8, kernel_size=3, stride=1, padding=1, bias=False)
#model.layer1[0].bn1 = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.layer1[0].bn2 = nn.Sequential()
#model.layer1[1].conv1 = QuantConv2d(8, 16, kernel_size=3, stride=1, padding=1, bias=False)
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): Sequential()
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        8, 8, kernel_size=(3, 3), strid

In [ ]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [25]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)


prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked


In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [14]:
PATH = "result/resnet20_quant4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8987/10000 (90%)



In [26]:
save_output.outputs[1][0].size()

torch.Size([128, 8, 32, 32])

In [222]:
w_bit = 4
weight_q = model.layer1[0].conv2.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv2.weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 0., -1., -1.],
          [-0., -2.,  3.],
          [-0., -2., -3.]],

         [[ 2., -2.,  2.],
          [-2., -2., -0.],
          [-0., -2., -3.]],

         [[-5., -1.,  7.],
          [ 1.,  1.,  7.],
          [-4.,  0.,  3.]],

         [[ 0., -2., -3.],
          [ 0., -3.,  0.],
          [ 0., -1., -1.]],

         [[-2., -2., -3.],
          [ 4., -7., -3.],
          [-0., -2., -3.]],

         [[ 1., -1., -2.],
          [-1.,  3., -2.],
          [ 3., -1., -1.]],

         [[ 1.,  1., -3.],
          [ 0., -5.,  6.],
          [ 1., -2., -0.]],

         [[ 2., -2., -1.],
          [ 1.,  4., -0.],
          [ 1., -0.,  1.]]],


        [[[-1.,  1.,  1.],
          [-1., -1., -1.],
          [-0., -3., -3.]],

         [[-1.,  2.,  1.],
          [ 0.,  1.,  5.],
          [-1.,  0., -0.]],

         [[-3.,  0., -2.],
          [-1.,  6., -0.],
          [-0.,  4.,  2.]],

         [[ 3.,  2.,  0.],
          [-2., -5., -5.],
          [ 0., -2., -0.]],

  

In [223]:
x_bit = 4
x = save_output.outputs[2][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv2.act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.,  0.,  0.,  ...,  2.,  2.,  1.],
          [ 2.,  1.,  1.,  ...,  0.,  0.,  0.],
          [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 1.,  1.,  1.,  ...,  1.,  2.,  1.],
          [ 1.,  2.,  2.,  ...,  2.,  2.,  1.]],

         [[ 0.,  0.,  0.,  ...,  2.,  2.,  2.],
          [ 0.,  1.,  0.,  ...,  0.,  0.,  1.],
          [ 0.,  0.,  0.,  ...,  1.,  1.,  2.],
          ...,
          [ 0.,  0.,  0.,  ...,  1.,  1.,  2.],
          [ 0.,  1.,  1.,  ...,  1.,  0.,  2.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  ...,  3.,  2.,  2.],
          [ 0.,  0.,  1.,  ...,  2.,  2.,  2.],
          [ 0.,  0.,  1.,  ...,  2.,  2.,  2.],
          ...,
          [ 6.,  2.,  2.,  ...,  2.,  2.,  2.],
          [ 7.,  2.,  2.,  ...,  2.,  2.,  1.],
          [ 4.,  3.,  2.,  ...,  2.,  2.,  2.]],

         ...,

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 2.,  3.,  

In [224]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
bn = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device)
output_int = conv_int(x_int)
output_int1 = output_int*w_delta*x_delta+save_output.outputs[1][0]
output_recovered = relu(output_int1)


In [225]:
difference = abs(save_output.outputs[3][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(3.4393e-07, device='cuda:0', grad_fn=<MeanBackward0>)


In [226]:
x_int.size()

torch.Size([128, 8, 32, 32])

In [227]:
X_pad = x_int[0]
print(X_pad.size())
x_test = X_pad[:,0,1:9]
print(x_test)


torch.Size([8, 32, 32])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [4., 4., 4., 3., 3., 3., 4., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 2., 1., 2., 0.],
        [4., 4., 4., 4., 4., 5., 3., 5.]], device='cuda:0',
       grad_fn=<SliceBackward>)


In [228]:
x_pad = torch.zeros(128, 8, 34, 34).cuda()

In [229]:
x_pad[ : ,  :, 1:33, 1:33] = x_int.cuda()
x_new= x_pad[:,:,0:3,0:10]
x_new.size()

torch.Size([128, 8, 3, 10])

In [230]:
X = x_new[0]
X = torch.reshape(X, (X.size(0), -1))

In [231]:
X.size()

torch.Size([8, 30])

In [234]:
bit_precision = 4
file = open('./resnet_out/activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [235]:
X[:,0]

tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [236]:
weight_int.size()

torch.Size([8, 8, 3, 3])

In [237]:
weight_int.size() # 8, 8 , 3, 3
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size() # 8, 8, 9

torch.Size([8, 8, 9])

In [238]:
bit_precision = 4
#file = open('weight.txt', 'w') #write to file
#file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
#file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
#file.write('#................#\n')
for kij in range(9):
    file = open('./resnet_out/w{}.txt'.format(str(kij)), 'w')
    file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
    file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
    file.write('#................#\n')
    for i in range(W.size(0)):  
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
    file.close() #close file  

In [239]:
W[0,:,0]

tensor([ 0.,  2., -5.,  0., -2.,  1.,  1.,  2.], device='cuda:0',
       grad_fn=<SelectBackward>)

In [240]:
output_int.size()

torch.Size([128, 8, 32, 32])

In [241]:
out = output_int[0]
out.size()
out = torch.reshape(out, (out.size(0), -1))
print(out.size())
out = out [:,0:8]
out.size()

torch.Size([8, 1024])


torch.Size([8, 8])

In [242]:
bit_precision = 16
file = open('./resnet_out/output.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):
    for j in range(out.size(0)):
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [243]:
out

tensor([[-60.,  -4.,  -2.,   0.,  13., -13.,  -7.,   9.],
        [  8.,  37.,  46.,  50.,  38.,  42.,  35.,  21.],
        [-19.,   2.,   5.,   5.,   7.,   7., -22.,  27.],
        [  0.,  -7.,  -8.,  -5.,  -1.,  21.,  -4., -16.],
        [ 19.,   2.,  -2.,  -5.,  -4.,   5.,   1.,  20.],
        [ 32., -16., -15., -16., -21., -26.,  16., -74.],
        [-21., -76., -70., -67., -55., -62., -70., -59.],
        [ 47.,  22.,  19.,  18.,   9.,  15.,   4.,  -3.]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [244]:
x_bit = 4
x0 = save_output.outputs[1][0]  # input of the 2nd conv layer
x0_alpha  = model.layer1[0].conv2.act_alpha
x0_delta = x0_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x0_q = act_quant_fn(x0, x0_alpha*w_alpha)         # create the quantized value for x

x0_int = x0_q/x0_delta/w_delta
print(x0_int)

tensor([[[[49.0000, 35.0000, 35.0000,  ...,  0.0000,  0.0000,  0.0000],
          [14.0000, 14.0000, 14.0000,  ..., 21.0000, 21.0000,  0.0000],
          [14.0000, 14.0000, 14.0000,  ..., 21.0000, 21.0000,  0.0000],
          ...,
          [21.0000, 21.0000, 21.0000,  ..., 21.0000, 21.0000,  0.0000],
          [21.0000, 21.0000, 21.0000,  ..., 21.0000, 21.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ..., 28.0000, 28.0000, 28.0000],
          [ 0.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000, 14.0000],
          [ 0.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000, 14.0000],
          ...,
          [21.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000, 14.0000],
          [21.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000, 14.0000],
          [14.0000,  7.0000,  7.0000,  ...,  7.0000,  7.0000,  7.0000]],

         [[49.0000, 14.0000, 14.0000,  ..., 14.0000, 14.0000, 49.0000],
          [49.0000,  7.0000,  

In [245]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
bn = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device)
output_int = conv_int(x_int)
output_int = (output_int+x0_int)*w_delta*x_delta
output_recovered = relu(output_int)
print(conv_int(x_int))

tensor([[[[ -60.,   -4.,   -2.,  ...,   -9.,  -31.,  -65.],
          [ -45.,  -34.,  -26.,  ...,  -24.,  -63.,  -84.],
          [ -48.,   -1.,   -6.,  ...,  -34.,  -64.,  -97.],
          ...,
          [  -8.,  -66.,  -24.,  ...,  -14.,  -66.,  -95.],
          [  10.,  -47.,  -19.,  ...,  -15.,  -60.,  -94.],
          [  18.,   -8.,   13.,  ...,    1.,  -26.,  -71.]],

         [[   8.,   37.,   46.,  ...,    9.,    4.,   35.],
          [  19.,   45.,   47.,  ...,   29.,   42.,   55.],
          [  20.,   25.,   48.,  ...,   27.,   28.,   49.],
          ...,
          [  92.,   47.,   45.,  ...,   19.,   17.,   52.],
          [  64.,    6.,    7.,  ...,  -16.,    0.,   37.],
          [   3.,  -31.,  -25.,  ...,  -20.,  -14.,   27.]],

         [[ -19.,    2.,    5.,  ...,  -19.,  -23.,   -5.],
          [ -21.,  -33.,  -28.,  ...,  -15.,  -33.,  -43.],
          [  15.,   -5.,  -11.,  ...,  -23.,  -21.,  -28.],
          ...,
          [  20.,  -19.,  -40.,  ...,    9.,  -15.,

In [246]:
difference = abs(save_output.outputs[3][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(0.1251, device='cuda:0', grad_fn=<MeanBackward0>)


In [247]:
x0_int.size()
X0 = x0_int[0]
X0 = torch.reshape(X0, (X0.size(0), -1))

In [248]:
X0.size()

torch.Size([8, 1024])

In [249]:
X0 = X0[:, 0:8]
X0.size()

torch.Size([8, 8])

In [250]:
bit_precision = 16
file = open('./resnet_out/residual.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X0.size(1)):  # time step
    for j in range(X0.size(0)): # row #
        X0_bin = '{0:016b}'.format(int(X0[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X0_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [251]:
X0

tensor([[49., 35., 35., 35., 35., 35., 35., 56.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [49., 14., 14., 14., 14., 14., 14.,  7.],
        [28., 14., 14., 14., 14., 14., 14., 14.],
        [ 0.,  7.,  7.,  0.,  0.,  0.,  0.,  0.],
        [77., 14., 14., 14., 14., 14., 14., 21.],
        [42., 35., 35., 35., 35., 35., 35., 35.],
        [ 7., 21., 21., 21., 28., 28., 21., 28.]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [252]:
answer = out + X0
print(answer)

tensor([[-11.,  31.,  33.,  35.,  48.,  22.,  28.,  65.],
        [  8.,  37.,  46.,  50.,  38.,  42.,  35.,  21.],
        [ 30.,  16.,  19.,  19.,  21.,  21.,  -8.,  34.],
        [ 28.,   7.,   6.,   9.,  13.,  35.,  10.,  -2.],
        [ 19.,   9.,   5.,  -5.,  -4.,   5.,   1.,  20.],
        [109.,  -2.,  -1.,  -2.,  -7., -12.,  30., -53.],
        [ 21., -41., -35., -32., -20., -27., -35., -24.],
        [ 54.,  43.,  40.,  39.,  37.,  43.,  25.,  25.]], device='cuda:0',
       grad_fn=<AddBackward0>)


In [253]:
bit_precision = 16
file = open('./resnet_out/answer.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(answer.size(1)):
    for j in range(answer.size(0)):
        if (answer[7-j,i].item()<0):
            ans_bin = '{0:016b}'.format(int(answer[7-j,i].item()+2**bit_precision+0.001))
        else:
            ans_bin = '{0:016b}'.format(int(answer[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(ans_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [266]:
f = open('./resnet_out/address.txt','r')
line = f.readlines()
bit_precision =11
file = open('./resnet_out/acc_address.txt','w')
for item in line:
    add = '{0:011b}'.format(int(item))
    for k in range(bit_precision):
        file.write(add[k])
    file.write('\n')
file.close()